# Module 5: Scale up your app

### Objectives

* Scale an app using kubectl.

### Scaling an application

In the previous modules we created a [Deployment](https://kubernetes.io/docs/concepts/workloads/controllers/deployment/), and then exposed it publicly via a [Service](https://kubernetes.io/docs/concepts/services-networking/service/). The Deployment created only one Pod for running our application. When traffic increases, we will need to scale the application to keep up with user demand.

**Scaling** is accomplished by changing the number of replicas in a Deployment

>You can create from the start a Deployment with multiple instances using the --replicas parameter for the kubectl create deployment command


## Scaling overview

<div><img src="https://d33wubrfki0l68.cloudfront.net/043eb67914e9474e30a303553d5a4c6c7301f378/0d8f6/docs/tutorials/kubernetes-basics/public/images/module_05_scaling1.svg" width="400">
<img src="https://d33wubrfki0l68.cloudfront.net/30f75140a581110443397192d70a4cdb37df7bfc/b5f56/docs/tutorials/kubernetes-basics/public/images/module_05_scaling2.svg" width="400">
</div>


Scaling out a Deployment will ensure new Pods are created and scheduled to Nodes with available resources. Scaling will increase the number of Pods to the new desired state. Kubernetes also supports [autoscaling](https://kubernetes.io/docs/user-guide/horizontal-pod-autoscaling/) of Pods, but it is outside of the scope of this tutorial. Scaling to zero is also possible, and it will terminate all Pods of the specified Deployment.

Running multiple instances of an application will require a way to distribute the traffic to all of them. Services have an integrated load-balancer that will distribute network traffic to all Pods of an exposed Deployment. Services will monitor continuously the running Pods using endpoints, to ensure the traffic is sent only to available Pods.

> Scaling is accomplished by changing the number of replicas in a Deployment.



Once you have multiple instances of an Application running, you would be able to do Rolling updates without downtime. We'll cover that in the next module. Now, let's go and scale our application.

### Lab 5 - Scaling up your app
#### Step 1 of 3: Scaling a deployment

To list your deployments use the get deployments command: kubectl get deployments

In [ ]:
studentId=$(grep hpecp-user $HOME/.kube/config | cut -d= -f2)
POD_NAME=$(kubectl get pods -o go-template --template '{% raw %}{{range .items}}{{.metadata.name}}{{"\n"}}{{end}}{% endraw %}' | grep $studentId) 
port=$(expr $(echo ${studentId} | tr -cd '[[:digit:]]') + 8000)

kubectl get deployments | grep $studentId


The output should be similar to:

````
NAME                  READY   UP-TO-DATE   AVAILABLE   AGE
kubernetes-bootcamp   1/1     1            1           11m
````

- *NAME* lists the names of the Deployments in the cluster.
- *READY* shows the ratio of CURRENT/DESIRED replicas
- *UP-TO-DATE* displays the number of replicas that have been updated to achieve the desired state.
- *AVAILABLE* displays how many replicas of the application are available to your users.
- *AGE* displays the amount of time that the application has been running.

To see the ReplicaSet created by the Deployment, run kubectl get rs



In [ ]:
kubectl get rs | grep $studentId

Notice that the name of the ReplicaSet is always formatted as [DEPLOYMENT-NAME]-[RANDOM-STRING]. The random string is randomly generated and uses the pod-template-hash as a seed.

Two important columns of this command are:

- *DESIRED* displays the desired number of replicas of the application, which you define when you create the Deployment. This is the desired state.
- *CURRENT* displays how many replicas are currently running.

Next, let’s scale the Deployment to 4 replicas. We’ll use the kubectl scale command, followed by the deployment type, name and desired number of instances:

In [ ]:
kubectl scale deployments/kubernetes-bootcamp-$studentId --replicas=4

To list your Deployments once again, use get deployments:

In [ ]:
kubectl get deployments | grep $studentId

The change was applied, and we have 4 instances of the application available. Next, let’s check if the number of Pods changed (retry this command multiple times until all pods are up)

In [ ]:
kubectl get pods -o wide | grep $studentId

There are 4 Pods now, with different IP addresses. The change was registered in the Deployment events log. To check that, use the describe command:

In [ ]:
kubectl describe deployments/kubernetes-bootcamp-$studentId

You can also view in the output of this command that there are 4 replicas now.

#### Step 2 of 3: Load Balancing

Let’s check that the Service is load-balancing the traffic. To find out the exposed IP and Port we can use the describe service as we learned in the previous Module:

In [ ]:
kubectl describe service/kubernetes-bootcamp-$studentId  

Create an environment variable called NODE_PORT that has the value of the Node port assigned.
And another one that compute NODE_NAME from a  `kubectl describe pods ` command. 

In [ ]:
NODE_PORT=$(kubectl get services/kubernetes-bootcamp-$studentId -o go-template='{% raw %}{{(index .spec.ports 0).nodePort}}{% endraw %}')
echo NODE_PORT=$NODE_PORT
NODE_NAME=$(kubectl describe pods $POD_NAME | grep Node:)
NODE_NAME=${NODE_NAME#Node:} 
NODE_NAME=${NODE_NAME%/*} 
NODE_NAME=$(echo $NODE_NAME | xargs)
echo NODE_NAME=$NODE_NAME


Next, we’ll do a `curl` to the exposed IP and port. Execute the command multiple times:

> To execute a multiple times, use CTRL-Enter while positioned on that cell.

In [ ]:
curl http://$NODE_NAME:$NODE_PORT

We hit a different Pod with every request. This demonstrates that the load-balancing is working.

You can monitor your application with the HPE Ezmeral Container Platform user interface: https://{{ JPHOSTEXT }}:{{ HPEECPK8S101PORT }}

Use student{{ STDID }}/{{ PASSSTU }} as your username/password to login and then select **K8s101** as your tenant in the upper right corner of the window, as shown below:

<img src="Pictures/k8s101webui.PNG">
 

#### Step 3 of 3: Deleting a service

To delete Services you can use the `delete service` command. Labels can be used also here:

In [ ]:
kubectl delete service -l app=kubernetes-bootcamp-$studentId

Confirm that the service is gone:

In [ ]:
kubectl get services | grep $studentId

This confirms that our Service was removed. To confirm that route is not exposed anymore you can curl the previously exposed IP and port:

In [ ]:
curl http://$NODE_NAME:$NODE_PORT

This proves that the app is not reachable anymore from outside of the cluster. You can confirm that the app is still running with a curl inside the pod:

In [ ]:
kubectl exec $POD_NAME -- curl -s localhost:8080

We see here that the application is up. This is because the Deployment is managing the application. To shut down the application, you would need to delete the Deployment as well.


This terminates module 5. Let's continue with [module 6, Update your app](./7-WKSHP-K8S101-mod6.ipynb).